In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 15, 10
rcParams["font.weight"] = "bold"
rcParams["axes.labelweight"] = "bold"
rcParams["font.size"] = 12

In [9]:
df_placas = pd.read_csv('placas_solares.csv')

df_placas

,Unnamed: 0,Unnamed: 0_level_0,2016,Unnamed: 3,2017,Unnamed: 5,2018,Unnamed: 7,2019,Unnamed: 9,2020,Unnamed: 11,2021,Unnamed: 13,2019.1,2021.1,Share of totalconsumption
0,NaN,Country or territory,New,Total,New,Total,New,Total,New,Total,New,Total,New,Total,W per capita 2019,W per capita 2021,Share of totalconsumption1
1,0.0,China,34540,78070,53000,131000,45000,175018,30100,204700,49655,254355,52618,306973,147,217,6.2% (2020)[23]
2,1.0,European Union,NaN,101433,5717,107150,8300,115234,16000,134129,18788,152917,25783,178700,295,400,6.0% (2020)[23]
3,2.0,United States,14730,40300,10600,51000,10600,53184,13300,60682,14890,75572,19637,95209,184,289,3.4% (2020)[23]
4,3.0,Japan,8600,42750,7000,49000,6500,55500,7000,63000,4000,67000,7191,74191,498,590,8.3% (2020)[23]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,87.0,Uzbekistan,NaN,2,NaN,3,NaN,4,NaN,4,NaN,4,NaN,104,NaN,3,NaN
89,88.0,Guatemala,NaN,93,NaN,99,NaN,101,NaN,101,NaN,101,NaN,101,NaN,6,NaN
90,89.0,Mali,NaN,18,NaN,19,NaN,19,NaN,19,NaN,70,NaN,100,NaN,5,NaN
91,90.0,Nepal,2,3,25,28,10,38,NaN,45,NaN,70,NaN,93,17,3,0.1% (2020)[23]


In [10]:
df_placas.isna().sum()

Unnamed: 0                    1
Unnamed: 0_level_0            0
2016                         54
Unnamed: 3                    4
2017                         51
Unnamed: 5                    5
2018                         60
Unnamed: 7                    0
2019                         65
Unnamed: 9                    0
2020                         63
Unnamed: 11                   0
2021                         66
Unnamed: 13                   0
2019.1                       31
2021.1                        2
Share of totalconsumption    47
dtype: int64

In [11]:
df_placas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 92 non-null     float64
 1   Unnamed: 0_level_0         93 non-null     object 
 2   2016                       39 non-null     object 
 3   Unnamed: 3                 89 non-null     object 
 4   2017                       42 non-null     object 
 5   Unnamed: 5                 88 non-null     object 
 6   2018                       33 non-null     object 
 7   Unnamed: 7                 93 non-null     object 
 8   2019                       28 non-null     object 
 9   Unnamed: 9                 93 non-null     object 
 10  2020                       30 non-null     object 
 11  Unnamed: 11                93 non-null     object 
 12  2021                       27 non-null     object 
 13  Unnamed: 13                93 non-null     object 
 

In [12]:
df_placas["2021"].isna()

0     False
1     False
2     False
3     False
4     False
      ...  
88     True
89     True
90     True
91     True
92     True
Name: 2021, Length: 93, dtype: bool

In [13]:
df_placas["2021"].isna().sum()

66

In [14]:
df_placas["2021"][df_placas["2021"].isna()]

26    NaN
27    NaN
28    NaN
29    NaN
30    NaN
     ... 
88    NaN
89    NaN
90    NaN
91    NaN
92    NaN
Name: 2021, Length: 66, dtype: object

In [17]:
df_placas[10:100][["2020", "2021", "2016"]]

,2020,2021,2016
10,10909,156,NaN
11,5378,1863,NaN
12,1833,2985,559
13,3488,4036,525
14,177,126,1970
...,...,...,...
88,NaN,NaN,NaN
89,NaN,NaN,NaN
90,NaN,NaN,NaN
91,NaN,NaN,2


In [18]:
df_placas.drop(df_placas.dropna().index)

,Unnamed: 0,Unnamed: 0_level_0,2016,Unnamed: 3,2017,Unnamed: 5,2018,Unnamed: 7,2019,Unnamed: 9,2020,Unnamed: 11,2021,Unnamed: 13,2019.1,2021.1,Share of totalconsumption
0,NaN,Country or territory,New,Total,New,Total,New,Total,New,Total,New,Total,New,Total,W per capita 2019,W per capita 2021,Share of totalconsumption1
2,1.0,European Union,NaN,101433,5717,107150,8300,115234,16000,134129,18788,152917,25783,178700,295,400,6.0% (2020)[23]
10,9.0,Vietnam,NaN,6,3,9,97,106,4800,5695,10909,16504,156,16660,60,171,NaN
11,10.0,Spain[27],NaN,4669,19,4688,19,4707,4004,8711,5378,14089,1863,15952,186,237,9.0% (2020)[23]
15,14.0,Brazil[28][29],NaN,200,900,1100,1313,2413,2138,4595,3145,7881,5827,"13,708[30]",22,64,2.5% (2021)[31]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,87.0,Uzbekistan,NaN,2,NaN,3,NaN,4,NaN,4,NaN,4,NaN,104,NaN,3,NaN
89,88.0,Guatemala,NaN,93,NaN,99,NaN,101,NaN,101,NaN,101,NaN,101,NaN,6,NaN
90,89.0,Mali,NaN,18,NaN,19,NaN,19,NaN,19,NaN,70,NaN,100,NaN,5,NaN
91,90.0,Nepal,2,3,25,28,10,38,NaN,45,NaN,70,NaN,93,17,3,0.1% (2020)[23]


In [20]:
df_placas.shape[0] - 54

39

In [21]:
df_placas_filt = df_placas.dropna().reset_index(drop=True)
df_placas_filt

,Unnamed: 0,Unnamed: 0_level_0,2016,Unnamed: 3,2017,Unnamed: 5,2018,Unnamed: 7,2019,Unnamed: 9,2020,Unnamed: 11,2021,Unnamed: 13,2019.1,2021.1,Share of totalconsumption
0,0.0,China,34540,78070,53000,131000,45000,175018,30100,204700,49655,254355,52618,306973,147,217,6.2% (2020)[23]
1,2.0,United States,14730,40300,10600,51000,10600,53184,13300,60682,14890,75572,19637,95209,184,289,3.4% (2020)[23]
2,3.0,Japan,8600,42750,7000,49000,6500,55500,7000,63000,4000,67000,7191,74191,498,590,8.3% (2020)[23]
3,4.0,Germany,1520,41220,1800,42000,3000,45930,3900,49200,4583,53783,4678,58461,593,702,9.7% (2020)[23]
4,5.0,India,3970,9010,9100,18300,10800,26869,9900,35089,4122,39211,10473,49684,32,36,6.5% (2020)[23]
5,6.0,Italy,373,19279,409,19700,420,20120,600,20800,800,21600,1098,22698,345,381,8.3% (2020)[23]
6,7.0,Australia,839,5900,1250,7200,3800,11300,3700,15928,1699,17627,1449,19076,637,742,10.7% (2020)[23]
7,8.0,South Korea,850,4350,1200,5600,2000,7862,3100,11200,3375,14575,3586,18161,217,350,3.8% (2020)[23]
8,11.0,France,559,7130,870,8000,1483,9483,417,9900,1833,11733,2985,14718,148,218,2.8% (2020)[23]
9,12.0,Netherlands,525,2100,853,2900,1300,4150,2575,6725,3488,10213,4036,14249,396,817,8.9% (2020)[23]
